In [113]:
import pandas as pd
from nltk import sent_tokenize, word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
import copy
import logging
from gensim.models import doc2vec
from gensim import matutils
from numpy import dot, array
import logging
import cPickle as pickle
import cProfile

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [41]:
with open('data/catalog.xlsx', 'r') as f:
	catalog = pd.read_excel(f) # type: pandas.core.frame.DataFrame
print catalog.head()

                                Title         Department  \
AFSC101      FOUNDATION OF THE USAF I  Air Force Science   
AFSC102     FOUNDATION OF THE USAF II  Air Force Science   
AFSC201      EVOLUTION OF AIR POWER I  Air Force Science   
AFSC202     EVOLUTION OF AIR POWER II  Air Force Science   
AFSC301  AIR FORCE LEADERSHIP STUDY I  Air Force Science   

                           Long Title  \
AFSC101      FOUNDATION OF THE USAF I   
AFSC102     FOUNDATION OF THE USAF II   
AFSC201      EVOLUTION OF AIR POWER I   
AFSC202     EVOLUTION OF AIR POWER II   
AFSC301  AIR FORCE LEADERSHIP STUDY I   

                                               Description  
AFSC101  Overall roles and missions of the USAF; career...  
AFSC102  Continuation of AFSC 101.  Course taught at th...  
AFSC201  Key historical events and milestones in the de...  
AFSC202  Continuation of AFSC 201.  Course taught at th...  
AFSC301  Leadership, management fundamentals, professio...  


In [42]:
# fill na
catalog = catalog.fillna('')

In [44]:
# extract data
catalog_train = catalog[['Long Title', 'Description']]

In [ ]:
def sentence_to_words(sentence, stop_word = False, stem = False):
	words = sent_tokenize(sentence)
	
	if stop_word == True:
		stop = stopwords.words('english')
		words = [w for w in words if w not in stop]
	
	return words

In [57]:
# tag sentence
train = []
for index, row in catalog_train.iterrows():
	for content in row:
		sentences = sent_tokenize(content)
		for sentence in sentences:
			train.append((sentence, index))
print 'Number of sentences: ', len(train)
# tokenize sentence
count = 0
for idx in range(len(train)):
	sentence = train[idx][0]
	tag = train[idx][1]
	words = word_tokenize(sentence)
	count += len(words)
	train[idx] = (words, tag)
print 'Number of words: ', count

Number of sentences:  24405


Number of words:  345022


In [59]:
# Create Tagged Document
documents = []
sentences = []
for ele in train:
	sent = ele[0]
	tag = [str(ele[1])]
	instance = doc2vec.TaggedDocument(sent, tag)    # type: doc2vec.TaggedDocument
	documents.append(instance)
	sentences.append(sent)

In [60]:
# Config Train Model
default_alpha = 0.025
min_alpha = 0.0001
windows = 8;

model = doc2vec.Doc2Vec(size=300, min_count = 3, window = windows, 
                        alpha= default_alpha, min_alpha= min_alpha)
model.build_vocab(documents)
model.intersect_word2vec_format('GoogleNews-vectors-negative300.bin', 
                                binary=True, lockf = 1.0)

In [73]:
# save model
with open('pickle/unlockModel','w') as f:
	model.save(f)

In [105]:
def train_epoch(m, epoch):
	"""
	Return a trained trainedmodel based on base trainedmodel & epoch time
	:param epoch:
	:return:
	"""
	model = copy.deepcopy(m)
	assert isinstance(model, doc2vec.Doc2Vec)
	
	total_examples = len(documents)
	model.train(documents, total_examples=total_examples, epochs=epoch)
	return model

In [115]:
dvmodel = train_epoch(model, 50)

2017-08-29 22:01:17,084 : INFO : training model with 3 workers on 8884 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=8


2017-08-29 22:01:18,103 : INFO : PROGRESS: at 1.53% examples, 202548 words/s, in_qsize 5, out_qsize 0


2017-08-29 22:01:19,139 : INFO : PROGRESS: at 3.05% examples, 197470 words/s, in_qsize 6, out_qsize 0


2017-08-29 22:01:20,168 : INFO : PROGRESS: at 4.46% examples, 191024 words/s, in_qsize 5, out_qsize 0


2017-08-29 22:01:21,188 : INFO : PROGRESS: at 6.14% examples, 197441 words/s, in_qsize 6, out_qsize 0


...

2017-08-29 22:02:18,974 : INFO : PROGRESS: at 98.91% examples, 210825 words/s, in_qsize 6, out_qsize 0


2017-08-29 22:02:19,538 : INFO : worker thread finished; awaiting finish of 2 more threads


2017-08-29 22:02:19,602 : INFO : worker thread finished; awaiting finish of 1 more threads


2017-08-29 22:02:19,606 : INFO : worker thread finished; awaiting finish of 0 more threads


2017-08-29 22:02:19,607 : INFO : training on 17251100 raw words (13191057 effective words) took 62.5s, 210993 effective words/s


In [108]:
def predict_course(model, query_list, n = 5):
	"""
	Given a pretrianed model and a list of query words, return the top-n related course with cosine similarity
	:param n: get top-n related course; default n = 5
	:param model: a doc2vec model
	:param query_list: a list of query word
	:type model: doc2vec.Doc2Vec
	:type query_list: list
	:return: [(tag, cosine_similarity)]
	"""
	dv = model.docvecs
	
	def get_vector(model, query_list):
		"""
		Return the sum vector of a list of words
		:type model: doc2vec.Doc2Vec
		:type query_list: list
		"""
		strlist = query_list.split(" ")
		v = [model[i] for i in strlist]
		return matutils.unitvec(array(v).sum(axis=0))
	
	sum_vec = get_vector(model, query_list)
	result = model.docvecs.most_similar([sum_vec], topn = n)
	return result

In [118]:
print predict_course(dvmodel, 'computer science')
print dvmodel.most_similar('computer science'.split())

[('HIST233', 0.27921849489212036), ('HIST533', 0.2791692018508911), ('STAT281', 0.23940983414649963), ('MUSI605', 0.23683518171310425), ('PSYC581', 0.2252344787120819)]
[(u'simulations', 0.40995684266090393), (u'physics', 0.3936920464038849), (u'biology', 0.3717336356639862), (u'hands-on', 0.3641414940357208), (u'mathematics', 0.36155372858047485), (u'laboratory', 0.3612132966518402), (u'engineering', 0.3518570065498352), (u'finance', 0.35036101937294006), (u'useful', 0.34551703929901123), (u'electronics', 0.34456297755241394)]
